**https://universe.roboflow.com/usety1975-ncrcr/de-r0ca9**

# Install Packages and Download Dataset

In [1]:
#Roboflow for Dataset
!pip install roboflow

#YOLOV8
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-man

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="ueSrp8zT1NmWHzCxoYmt")
project = rf.workspace("usety1975-ncrcr").project("de-r0ca9")
version = project.version(3)
dataset = version.download("yolov8-obb")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to de-3 in yolov8-obb:: 100%|██████████| 20024/20024 [00:05<00:00, 3663.84it/s]


In [3]:
import ultralytics

# Check GPU
ultralytics.checks()

Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.4/107.7 GB disk)


# Fix YAML Paths

In [4]:
import yaml
import os

def edit_yaml_paths(yaml_file_path):
    # Dataset main directory
    root_dir = os.path.dirname(yaml_file_path)

    # Load the YAML file
    with open(yaml_file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Modify YAML file to include correct subset paths
    data['train'] = os.path.join(root_dir, 'train/images')
    data['val'] = os.path.join(root_dir, 'valid/images')
    data['test'] = os.path.join(root_dir, 'test/images')

    with open(yaml_file_path, 'w') as file:
        yaml.safe_dump(data, file)

    print("YAML file updated successfully!")


In [5]:
yaml_path = '/content/de-3/data.yaml'
edit_yaml_paths(yaml_path)

YAML file updated successfully!


# Create, Train, and Validate Model

In [7]:
from ultralytics import YOLO
import warnings
# Load trained model
weights_path = '/content/best.pt'
try:
    object_detect_model = YOLO(weights_path)
    print(f"Loaded model with best weights from {weights_path}")
except FileNotFoundError:
    warnings.warn(f"MODEL NOT FOUND AT {weights_path}, LOADING GENERIC PRETRAINED YOLOv8 MODEL INSTEAD.")
    object_detect_model = YOLO('yolov8m-seg.pt')  # Load a normal YOLO model with pretrained weights

MODEL NOT FOUND AT /content/best.pt, LOADING GENERIC PRETRAINED YOLOv8 MODEL INSTEAD.
100%|██████████| 52.4M/52.4M [00:00<00:00, 146MB/s]


In [ ]:
# TRAIN, VALIDATE, AND TEST MODEL
training = object_detect_model.train(data=yaml_path, epochs=15)
validation = object_detect_model.val()
testing = object_detect_model.val(data=yaml_path, split='test')

Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/de-3/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /content/de-3/train/labels... 8777 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8777/8777 [00:10<00:00, 821.34it/s]


train: New cache created: /content/de-3/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/de-3/valid/labels... 1156 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1156/1156 [00:01<00:00, 964.55it/s]


val: New cache created: /content/de-3/valid/labels.cache
Plotting labels to runs/segment/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000217, momentum=0.9) with parameter groups 86 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/train2
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/549 [00:00<?, ?it/s]

# Zip weights folder to be downloaded

In [ ]:
import shutil
from IPython.display import FileLink


directory_to_zip = "/content/runs"
output_zip = "/content/runs.zip"

# Zip the directory
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', directory_to_zip)

# Check if the zip file was created successfully
if os.path.exists(output_zip):
    print(f"Successfully created zip file: {output_zip}")
    # Provide a link to download the zip file
    display(FileLink(output_zip))
else:
    print("Failed to create zip file")


Successfully created zip file: /content/runs.zip


/content/runs.zip